## Project Setup 

In [1]:
import pandas as pd
import numpy as np
import os
import json
import random
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt  
import seaborn as sns
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler

In [2]:
# CONFIGURATION & CONSTANTS

# seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

LBS_TO_KG = 0.453592 # conversion factor from pounds to kilograms
HYPEROPT_N_TRIALS = 20 # number of hyperparameter optimization trials


## Data Loading and Preparation Functions

In [3]:
def load_data():

    df_rec = pd.read_csv('../data/kernel/receivals.csv')
    df_porder = pd.read_csv('../data/kernel/purchase_orders.csv')
    df_materials = pd.read_csv('../data/extended/materials.csv')
    prediction_mapping = pd.read_csv('../data/prediction_mapping.csv')

    # Data Cleaning and Normalization
    df_rec = df_rec.dropna(subset=['date_arrival', 'rm_id', 'net_weight'])
    df_porder = df_porder.dropna(subset=['delivery_date', 'product_id', 'quantity'])

    df_porder['delivery_date'] = pd.to_datetime(df_porder['delivery_date'], utc=True, errors='coerce').dt.date
    df_rec['date_arrival'] = pd.to_datetime(df_rec['date_arrival'], utc=True, errors='coerce').dt.date
    prediction_mapping['forecast_start_date'] = pd.to_datetime(prediction_mapping['forecast_start_date'], errors='coerce').dt.date
    prediction_mapping['forecast_end_date']   = pd.to_datetime(prediction_mapping['forecast_end_date'], errors='coerce').dt.date
    
    mask_p = (df_porder['unit_id'] == 43) 
    df_porder.loc[mask_p, 'quantity'] = LBS_TO_KG * df_porder.loc[mask_p, 'quantity']
    df_porder.loc[mask_p, 'unit_id'] = 40 
    df_porder.loc[mask_p, 'unit'] = 'kg'
    
    df_rec = df_rec.loc[df_rec['net_weight'] > 0].copy()
    df_porder = df_porder.loc[df_porder['quantity'] > 0].copy()

    return df_rec, df_porder, df_materials, prediction_mapping


def aggregates_3df_daily(df_rec, df_porder, df_materials):
    # print("Aggregating receivals and purchase orders dataframes on a daily level on rm_id")

    receivals_daily_agg = df_rec.groupby(['rm_id', 'date_arrival']).net_weight.sum().reset_index()
    map_product_rm = df_materials[['rm_id', 'product_id']].drop_duplicates().dropna()

    df_porder_w_rmid = pd.merge(df_porder, map_product_rm, on='product_id', how='left').dropna(subset=['rm_id'])
    daily_po_agg = df_porder_w_rmid.groupby(['rm_id', 'delivery_date']).quantity.sum().reset_index()
    daily_po_agg['rm_id'] = daily_po_agg['rm_id'].astype(int)
    receivals_daily_agg['rm_id'] = receivals_daily_agg['rm_id'].astype(int)
    return receivals_daily_agg, daily_po_agg


## Forecasting Using Croston

In [4]:
def _croston_forecast(y, h=151, alpha=0.1):

    y = np.asarray(y, dtype=float)
    demand = y[y > 0.0]
    if demand.size == 0:
        return np.zeros(h, dtype=float)
    
    intervals = np.diff(np.concatenate(([0], np.where(y > 0)[0])))
    z = demand[0]
    p = intervals[0] if intervals.size > 0 else 1.0
    
    for k in range(1, demand.size):
        z = alpha * demand[k]   + (1 - alpha) * z
    for k in range(1, intervals.size):
        p = alpha * intervals[k] + (1 - alpha) * p
        
    rate = z / max(p, 1e-6) 
    return np.full(h, rate, dtype=float)


def build_intermittent_baseline(daily_receivals, start_date, end_date, method="croston", alpha=0.1):

    horizon_days = (end_date - start_date).days + 1
    daily_receivals_copy = daily_receivals.copy()
    daily_receivals_copy['date_arrival'] = pd.to_datetime(daily_receivals_copy['date_arrival'])
    
    # Build full daily panel per rm_id
    result = []
    for rm, group in daily_receivals_copy.groupby('rm_id'):
        group = group.set_index('date_arrival').sort_index()
        daily_series = group['net_weight'].asfreq('D', fill_value=0.0)
        
        # Use last N years only to avoid ancient regimes
        cutoff = pd.Timestamp(start_date) - pd.Timedelta(days=365*3)
        history_series = daily_series[daily_series.index < pd.Timestamp(start_date)]
        
        if history_series.index.min() is None:
            history_series = daily_series[daily_series.index < pd.Timestamp(start_date)]
            
        history_series = history_series[history_series.index >= cutoff]
        
        if history_series.empty:
            forecast = np.zeros(horizon_days, dtype=float)
        else:
            if method == "croston":
                forecast = _croston_forecast(history_series.values, h=horizon_days, alpha=alpha)
            else:
                forecast = _croston_forecast(history_series.values, h=horizon_days, alpha=alpha) 
                
        dates = pd.date_range(start_date, end_date, freq='D')
        result.append(pd.DataFrame({'rm_id': rm, 'date': dates, 'baseline_daily_fc': forecast}))
        
    return pd.concat(result, ignore_index=True)


def sum_baseline_over_windows(features_df, baseline_df):

    original_index = features_df.index
    rm_ids = features_df['rm_id'].values
    start_dates = pd.to_datetime(features_df['forecast_start_date']).values.astype('datetime64[D]')
    end_dates   = pd.to_datetime(features_df['forecast_end_date']).values.astype('datetime64[D]')
    
    # here we build map rm_id -> arrays for vectorized summation
    baseline_lookup_map = {}
    for k, g in baseline_df.groupby('rm_id'):
        dates = pd.to_datetime(g['date']).values.astype('datetime64[D]')
        values = g['baseline_daily_fc'].values.astype(float)
        baseline_lookup_map[k] = (dates, np.cumsum(values))
        
    output_sums = np.zeros(len(features_df), dtype=float)
    
    for i in range(len(features_df)):
        lookup_tuple = baseline_lookup_map.get(rm_ids[i])
        if lookup_tuple is None: 
            continue
            
        dates, cumulative_values = lookup_tuple
        left_idx = np.searchsorted(dates, start_dates[i], side='left')
        right_idx = np.searchsorted(dates, end_dates[i],   side='right') - 1
        
        if right_idx >= left_idx and left_idx < len(cumulative_values):
            left_sum = cumulative_values[left_idx-1] if left_idx > 0 else 0.0
            output_sums[i] = cumulative_values[right_idx] - left_sum
            
    return pd.Series(output_sums, index=original_index, name='baseline_sum_in_window')


## Feature Engineering Functions

In [5]:
def create_features(input_df, daily_receivals, daily_po, receivals_full=None, purchase_orders_full=None):

    index_original = input_df.index.name
    features_df = input_df.reset_index()
    if 'index' not in features_df.columns and index_original is not None:
        features_df = features_df.rename(columns={index_original: 'index'})

    features_df['window_length'] = (
        features_df['forecast_end_date'] - features_df['forecast_start_date']
    ).apply(lambda x: x.days + 1).astype(np.int16)

    features_df['end_month'] = features_df['forecast_end_date'].apply(lambda x: x.month).astype(np.int16)
    features_df['end_dayofweek'] = features_df['forecast_end_date'].apply(lambda x: x.weekday()).astype(np.int16)

    features_df['month_sin'] = np.sin(2 * np.pi * features_df['end_month'] / 12).astype(np.float32)
    features_df['month_cos'] = np.cos(2 * np.pi * features_df['end_month'] / 12).astype(np.float32)

    start_dates_np = pd.to_datetime(features_df["forecast_start_date"]).values.astype('datetime64[D]')
    end_dates_np   = pd.to_datetime(features_df["forecast_end_date"]).values.astype('datetime64[D]')
    features_df["business_days_in_window"] = [np.busday_count(s, e + np.timedelta64(1, 'D')) for s, e in zip(start_dates_np, end_dates_np)]
    features_df["business_days_in_window"] = features_df["business_days_in_window"].astype(np.int16)
    features_df["weekends_in_window"] = (features_df["window_length"] - features_df["business_days_in_window"]).astype(np.int16)

    end_timestamps = pd.to_datetime(features_df["forecast_end_date"])
    features_df["end_days_to_month_end"] = (end_timestamps.dt.days_in_month - end_timestamps.dt.day).astype(np.int16)

    #   PO (Purchase Order) in window  
    pp = daily_po.copy()
    # datetime check
    if not np.issubdtype(pp['delivery_date'].dtype, np.datetime64):
        pp['delivery_date'] = pd.to_datetime(pp['delivery_date'], errors='coerce')

    po_by_rm = {rm: grp.sort_values('delivery_date').reset_index(drop=True)
                for rm, grp in pp.groupby('rm_id')}

    po_in_window_vals = np.zeros(len(features_df), dtype=np.float32)

    for rm, rows in features_df.groupby('rm_id'):
        po_grp = po_by_rm.get(rm)
        if po_grp is None or po_grp.empty:
            continue

        row_positions = rows.index.values
        po_dates = po_grp['delivery_date'].values.astype('datetime64[D]')
        po_qty = po_grp['quantity'].values.astype(float)
        po_cum = np.cumsum(po_qty)

        starts = pd.to_datetime(rows['forecast_start_date']).values.astype('datetime64[D]')
        ends   = pd.to_datetime(rows['forecast_end_date']).values.astype('datetime64[D]')

        left_pos  = np.searchsorted(po_dates, starts, side='left')
        right_pos = np.searchsorted(po_dates, ends,   side='right') - 1

        # sums for cumulative
        sums = np.zeros(len(row_positions), dtype=float)
        for i, (l, r) in enumerate(zip(left_pos, right_pos)):
            if r >= l and r >= 0 and l < len(po_cum):
                left_cum = po_cum[l-1] if l > 0 else 0.0
                sums[i] = po_cum[r] - left_cum
            else:
                sums[i] = 0.0
        po_in_window_vals[row_positions] = sums

    features_df['po_in_window'] = po_in_window_vals
    features_df['daily_avg_po_in_window'] = (
        features_df['po_in_window'] / features_df['window_length']
    ).replace([np.inf, -np.inf], 0).fillna(0).astype(np.float32)

    # Historical aggregates  
    historical_aggregates_list = []
    rec_horizons = [7, 30, 90]
    po_horizons  = [7, 30, 90]
    rec_rolling_windows = [30, 90]

    dr = daily_receivals.copy()
    dr['date_arrival'] = pd.to_datetime(dr['date_arrival'], errors='coerce')

    for start_date in features_df['forecast_start_date'].unique():
        reference_date = pd.to_datetime(start_date) - pd.Timedelta(days=1)
        rec_frames, po_frames = [], []

        for h in rec_horizons:
            hist_start = reference_date - pd.Timedelta(days=(h - 1))
            hist_rec = dr[(dr['date_arrival'] >= hist_start) & (dr['date_arrival'] <= reference_date)]
            rec_agg = hist_rec.groupby('rm_id')['net_weight'].sum()
            rec_agg.name = f'hist_rec_{h}d'
            rec_frames.append(rec_agg)

        for h in po_horizons:
            hist_start = reference_date - pd.Timedelta(days=(h - 1))
            hist_po = daily_po[
                (pd.to_datetime(daily_po['delivery_date']) >= hist_start) &
                (pd.to_datetime(daily_po['delivery_date']) <= reference_date)
            ]
            po_agg = hist_po.groupby('rm_id')['quantity'].sum()
            po_agg.name = f'hist_po_{h}d'
            po_frames.append(po_agg)

        for window in rec_rolling_windows:
            hist_start = reference_date - pd.Timedelta(days=(window - 1))
            historical_receivals = dr[(dr['date_arrival'] >= hist_start) & (dr['date_arrival'] <= reference_date)]
            
            rec_mean = historical_receivals.groupby('rm_id')['net_weight'].mean()
            rec_mean.name = f'rec_mean_{window}d'
            
            rec_std = historical_receivals.groupby('rm_id')['net_weight'].std()
            rec_std.name = f'rec_std_{window}d'
            
            def _trend(g):
                if len(g) < 2: return 0.0
                t = (g['date_arrival'] - g['date_arrival'].min()).dt.days.values
                if np.unique(t).size < 2: return 0.0
                return float(np.polyfit(t, g['net_weight'].values, 1)[0])
            
            # future warnings handle
            if historical_receivals.empty:
                 rec_trend = pd.Series(dtype=float, name=f'rec_trend_{window}d')
                 rec_trend.index.name = 'rm_id'
            else:
                 try:
                     trend_result = historical_receivals.groupby('rm_id').apply(_trend, include_groups=False)
                 except TypeError:
                     trend_result = historical_receivals.groupby('rm_id').apply(_trend)
                 if isinstance(trend_result, pd.DataFrame):
                     rec_trend = trend_result.iloc[:, 0]
                 else:
                     rec_trend = trend_result
                 rec_trend.name = f'rec_trend_{window}d'
            
            rec_cnt = historical_receivals.groupby('rm_id')['date_arrival'].nunique()
            rec_cnt.name = f'rec_count_{window}d'
            
            rec_frames.extend([rec_mean, rec_std, rec_trend, rec_cnt])
        
        all_aggregates = pd.concat(rec_frames + po_frames, axis=1)
        all_aggregates = all_aggregates.fillna(0).reset_index()
        
        all_aggregates['forecast_start_date'] = start_date
        historical_aggregates_list.append(all_aggregates)


    all_hist_agg = pd.concat(historical_aggregates_list, ignore_index=True)
    features_df = features_df.merge(all_hist_agg, on=['rm_id', 'forecast_start_date'], how='left').fillna(0)

    features_df['po_to_rec_ratio_30d'] = np.where(features_df['hist_rec_30d'] > 0,
                                               features_df['hist_po_30d'] / features_df['hist_rec_30d'], 0).astype(np.float32)
    features_df['rec_cv_30d'] = np.where(features_df['rec_mean_30d'] > 0,
                                         features_df['rec_std_30d'] / features_df['rec_mean_30d'], 0).astype(np.float32)
    features_df['rec_zero_ratio_30d'] = (1.0 - (features_df.get('rec_count_30d', 0) / 30.0)).astype(np.float32)

    # YoY same-window 
    rec_by_rm = {rm: g.sort_values('date_arrival').reset_index(drop=True)
                 for rm, g in dr.groupby('rm_id')}
    last_year_vals = np.zeros(len(features_df), dtype=float)
    starts_ly = pd.to_datetime(features_df["forecast_start_date"]) - pd.Timedelta(days=365)
    ends_ly   = pd.to_datetime(features_df["forecast_end_date"])   - pd.Timedelta(days=365)

    for rm, rows in features_df.groupby('rm_id'):
        r = rec_by_rm.get(rm)
        if r is None or r.empty:
            continue
        dates = r['date_arrival'].values.astype('datetime64[D]')
        w = r['net_weight'].values.astype(float)
        c = np.cumsum(w)
        L = np.searchsorted(dates, starts_ly.loc[rows.index].values.astype('datetime64[D]'), side='left')
        R = np.searchsorted(dates, ends_ly.loc[rows.index].values.astype('datetime64[D]'),   side='right') - 1
        s = np.zeros(len(rows), dtype=float)
        for i, (l, rpos) in enumerate(zip(L, R)):
            if rpos >= l and l < len(c):
                s[i] = c[rpos] - (c[l-1] if l > 0 else 0.0)
        last_year_vals[rows.index.values] = s

    features_df["last_year_weight"] = last_year_vals.astype(np.float32)

    # Seasonal priors
    # print("  Computing rm seasonal priors (global baseline)")
    dr['month'] = dr['date_arrival'].dt.month
    dr['dow']   = dr['date_arrival'].dt.weekday

    mprior = dr.groupby(["rm_id", "month"])["net_weight"].mean()
    mprior.name = "rm_month_mean_prior"
    
    dprior = dr.groupby(["rm_id", "dow"])["net_weight"].mean()
    dprior.name = "rm_dow_mean_prior"
    
    gprior = dr.groupby("month")["net_weight"].mean()
    gprior.name = "global_month_mean_prior"

    features_df = features_df.merge(mprior.reset_index(),
                                   left_on=["rm_id", "end_month"], right_on=["rm_id", "month"], how="left").drop(columns=["month"])
    features_df = features_df.merge(dprior.reset_index(),
                                   left_on=["rm_id", "end_dayofweek"], right_on=["rm_id", "dow"], how="left").drop(columns=["dow"])
    features_df = features_df.merge(gprior.reset_index().rename(columns={"month": "end_month"}),
                                   on="end_month", how="left")

    features_df["rm_month_mean_prior"] = features_df["rm_month_mean_prior"].fillna(0.0)
    features_df["rm_dow_mean_prior"]   = features_df["rm_dow_mean_prior"].fillna(0.0)
    features_df["global_month_mean_prior"] = features_df["global_month_mean_prior"].fillna(0.0)
    features_df["rm_month_vs_global"] = (features_df["rm_month_mean_prior"] - features_df["global_month_mean_prior"]).astype(np.float32)

    # typical load size and expected truck count
    load_stats = (dr.groupby(["rm_id", "date_arrival"])["net_weight"].sum()
                      .reset_index()
                      .query("net_weight > 0"))
    
    typical_load = load_stats.groupby("rm_id")["net_weight"].median()
    typical_load.name = "typical_load_rm"
    
    features_df = features_df.merge(typical_load.reset_index(), on="rm_id", how="left")
    global_median_load = float(load_stats["net_weight"].median()) if len(load_stats) else 0.0
    features_df["typical_load_rm"] = features_df["typical_load_rm"].fillna(global_median_load).astype(np.float32)
    features_df["expected_truck_count"] = (features_df["po_in_window"] / (features_df["typical_load_rm"] + 1e-6)).clip(lower=0).astype(np.float32)

    # true lead-time and supplier reliability
    if (receivals_full is not None) and (purchase_orders_full is not None):
        # print("  Calculating: True lead-time and supplier reliability  ")
        rf = receivals_full.copy()
        pf = purchase_orders_full.copy()

        rf["date_arrival"]  = pd.to_datetime(rf["date_arrival"], errors="coerce")
        pf["delivery_date"] = pd.to_datetime(pf["delivery_date"], errors="coerce")

        join_keys = []
        if "purchase_order_id" in rf.columns and "purchase_order_item_no" in rf.columns:
            join_keys = ["purchase_order_id", "purchase_order_item_no"]
        elif "purchase_order_id" in rf.columns:
            join_keys = ["purchase_order_id"]

        if join_keys:
            jt = rf.merge(
                pf[join_keys + [c for c in ["product_id", "delivery_date", "quantity"] if c in pf.columns]],
                on=join_keys, how="left", suffixes=("", "_po")
            )
            if "product_id_po" in jt.columns and "product_id" in jt.columns:
                mask_pid = jt["product_id_po"].isna() | (jt["product_id_po"] == jt["product_id"])
                jt = jt[mask_pid]

            jt = jt.dropna(subset=["date_arrival", "delivery_date"])
            jt["lead_time_days"] = (jt["date_arrival"] - jt["delivery_date"]).dt.days
            jt = jt[(jt["lead_time_days"] >= -10) & (jt["lead_time_days"] <= 180)]

            lt_rm = jt.groupby("rm_id")["lead_time_days"].agg(["mean", "std", "median"]).reset_index()
            lt_rm.columns = ["rm_id", "lead_time_mean_rm", "lead_time_std_rm", "lead_time_median_rm"]
            features_df = features_df.merge(lt_rm, on="rm_id", how="left")

            # supplier reliability 
            if "supplier_id" in rf.columns:
                lt_sup = jt.groupby("supplier_id")["lead_time_days"].agg(["mean", "std", "median"]).reset_index()
                lt_sup.columns = ["supplier_id", "sup_lead_mean", "sup_lead_std", "sup_lead_median"]

                dom_sup = (rf.groupby(["rm_id", "supplier_id"])["net_weight"].sum()
                             .reset_index()
                             .sort_values(["rm_id", "net_weight"], ascending=[True, False])
                             .drop_duplicates(["rm_id"])
                             .rename(columns={"supplier_id": "dominant_supplier_id"}))[["rm_id", "dominant_supplier_id"]]

                features_df = features_df.merge(dom_sup, on="rm_id", how="left")
                features_df = features_df.merge(lt_sup, left_on="dominant_supplier_id", right_on="supplier_id", how="left")
                if "supplier_id" in features_df.columns:
                    features_df.drop(columns=["supplier_id"], inplace=True)

                features_df["supplier_reliability_score"] = (
                    (-features_df["sup_lead_mean"].fillna(0.0) / 30.0) +
                    (-features_df["sup_lead_std"].fillna(0.0)  / 15.0)
                ).astype(np.float32)

            for c in ["lead_time_mean_rm", "lead_time_std_rm", "lead_time_median_rm",
                      "sup_lead_mean", "sup_lead_std", "sup_lead_median", "supplier_reliability_score"]:
                if c in features_df.columns:
                    features_df[c] = features_df[c].fillna(0.0).astype(np.float32)
        else:
            for c in ["lead_time_mean_rm", "lead_time_std_rm", "lead_time_median_rm",
                      "sup_lead_mean", "sup_lead_std", "sup_lead_median", "supplier_reliability_score"]:
                features_df[c] = 0.0 

    #   Croston forecasting as features  
    win_min = pd.to_datetime(features_df['forecast_start_date']).min()
    win_max = pd.to_datetime(features_df['forecast_end_date']).max()
    
    if pd.isna(win_min) or pd.isna(win_max):
        features_df['baseline_sum_in_window'] = 0.0
    else:
        baseline_df = build_intermittent_baseline(daily_receivals, win_min, win_max, method="croston", alpha=0.1)
        features_df['baseline_sum_in_window'] = sum_baseline_over_windows(features_df, baseline_df).astype(np.float32)

    core_feature_cols = [
        'rm_id',
        'window_length',

        # calendar
        'end_dayofweek', 'end_days_to_month_end',
        'business_days_in_window', 'weekends_in_window',
        'month_sin', 'month_cos',
        'baseline_sum_in_window',

        # PO window
        'po_in_window', 'daily_avg_po_in_window',

        # historical sums
        'hist_rec_7d', 'hist_rec_30d', 'hist_rec_90d',
        'hist_po_7d',  'hist_po_30d',  'hist_po_90d',

        # rolling stats and trend
        'rec_mean_30d', 'rec_std_30d', 'rec_trend_30d',
        'rec_mean_90d', 'rec_std_90d', 'rec_trend_90d',

        # counts and sparsity
        'rec_count_30d', 'rec_count_90d', 'rec_zero_ratio_30d',

        # ratios
        'po_to_rec_ratio_30d', 'rec_cv_30d',

        # priors & YoY
        'last_year_weight', 'rm_month_mean_prior', 'rm_dow_mean_prior', 'rm_month_vs_global',

        # logistics
        'typical_load_rm', 'expected_truck_count',

        # ead-time and supplier reliability
        'lead_time_mean_rm', 'lead_time_std_rm', 'lead_time_median_rm',
        'supplier_reliability_score'
    ]

    final_feature_cols = core_feature_cols + ['index', 'forecast_start_date', 'forecast_end_date']
    
    # if skip then fill with zeross
    for col in core_feature_cols:
        if col not in features_df.columns:
            features_df[col] = 0.0
            
    return features_df[final_feature_cols].set_index('index')

    
def training_data(daily_receivals, daily_po, year_for_training, receivals_full=None, purchase_orders_full=None):

    windows_for_train = []
    all_rm_ids = daily_receivals['rm_id'].unique()
    for rm_id in all_rm_ids:
        for year in year_for_training:
            start_date = date(year, 1, 1)
            end_max = date(year, 5, 31)
            max_horizon = (end_max - start_date).days  
            for h in range(0, max_horizon + 1): 
                end_date = start_date + timedelta(days=h)
                windows_for_train.append({'rm_id': rm_id, 'forecast_start_date': start_date, 'forecast_end_date': end_date})
    raw_train = pd.DataFrame(windows_for_train)

    X_train = create_features(raw_train, daily_receivals, daily_po,
                              receivals_full=receivals_full, purchase_orders_full=purchase_orders_full)
 
    # target 
    y_train = pd.Series(0.0, index=X_train.index, name='target', dtype=float)
    daily_receivals_copy = daily_receivals.copy()
    if not np.issubdtype(daily_receivals_copy['date_arrival'].dtype, np.datetime64):
        daily_receivals_copy['date_arrival'] = pd.to_datetime(daily_receivals_copy['date_arrival'], errors='coerce')
    receivals_lookup_by_rm = {rm: grp.sort_values('date_arrival').reset_index(drop=True)
                              for rm, grp in daily_receivals_copy.groupby('rm_id')}

    for rm_id, rows in X_train.reset_index().groupby('rm_id'):
        rows_index = rows['index'].values
        receivals_group = receivals_lookup_by_rm.get(rm_id)
        if receivals_group is None or receivals_group.empty: continue
        
        rec_dates = receivals_group['date_arrival'].values.astype('datetime64[D]')
        rec_weights = receivals_group['net_weight'].values.astype(float)
        rec_cumulative = np.cumsum(rec_weights)
        starts = pd.to_datetime(rows['forecast_start_date']).values.astype('datetime64[D]')
        ends = pd.to_datetime(rows['forecast_end_date']).values.astype('datetime64[D]')
        left_pos = np.searchsorted(rec_dates, starts, side='left')
        right_pos = np.searchsorted(rec_dates, ends, side='right') - 1

        sums = np.zeros(len(rows_index), dtype=float)
        for i, (l, r) in enumerate(zip(left_pos, right_pos)):
            if r >= l and r >= 0 and l < len(rec_cumulative):
                left_cum = rec_cumulative[l-1] if l > 0 else 0.0
                sums[i] = rec_cumulative[r] - left_cum
            else:
                sums[i] = 0.0 
        y_train.loc[rows_index] = sums

    X_train_export = X_train.drop(columns=['forecast_start_date', 'forecast_end_date'], errors='ignore')
    return X_train_export, y_train

def run_data_preparation():

    loaded_data = load_data()
    receivals, purchase_orders, materials, prediction_mapping = loaded_data
    daily_receivals, daily_po = aggregates_3df_daily(receivals, purchase_orders, materials)

    min_year = receivals['date_arrival'].min().year
    max_year = 2024
    all_years = list(range(min_year, max_year + 1))
    covid_years = {2020, 2021, 2022} #covid years
    training_years = [year for year in all_years if year not in covid_years]

    
    X_train, y_train = training_data(
        daily_receivals, daily_po, year_for_training=training_years,
        receivals_full=receivals, purchase_orders_full=purchase_orders
    )

    # print("Exporting training dataset  ")
    training_data_to_export = X_train.copy()
    training_data_to_export['target'] = y_train
    return training_data_to_export


## Training Data Generation Functions

In [6]:
def training_data(daily_receivals, daily_po, year_for_training, receivals_full=None, purchase_orders_full=None):

    windows_for_train = []
    all_rm_ids = daily_receivals['rm_id'].unique()

    for rm_id in all_rm_ids:
        for year in year_for_training:
            start_date = date(year, 1, 1)
            end_max = date(year, 5, 31)
            max_horizon = (end_max - start_date).days  
            
            for h in range(0, max_horizon + 1): 
                end_date = start_date + timedelta(days=h)
                windows_for_train.append({
                    'rm_id': rm_id,
                    'forecast_start_date': start_date,
                    'forecast_end_date': end_date
                })

    raw_train = pd.DataFrame(windows_for_train)

    # Build features
    X_train = create_features(raw_train, daily_receivals, daily_po, receivals_full=receivals_full, purchase_orders_full=purchase_orders_full)

    y_train = pd.Series(0.0, index=X_train.index, name='target', dtype=float)
    daily_receivals_copy = daily_receivals.copy()
     
    if not np.issubdtype(daily_receivals_copy['date_arrival'].dtype, np.datetime64): # datetime check
        daily_receivals_copy['date_arrival'] = pd.to_datetime(daily_receivals_copy['date_arrival'], errors='coerce')
        
    receivals_lookup_by_rm = {rm: grp.sort_values('date_arrival').reset_index(drop=True)
                              for rm, grp in daily_receivals_copy.groupby('rm_id')}

    for rm_id, rows in X_train.reset_index().groupby('rm_id'):
        rows_index = rows['index'].values
        receivals_group = receivals_lookup_by_rm.get(rm_id)
        
        if receivals_group is None or receivals_group.empty:
            continue
            
        rec_dates = receivals_group['date_arrival'].values.astype('datetime64[D]')
        rec_weights = receivals_group['net_weight'].values.astype(float)
        rec_cumulative = np.cumsum(rec_weights)
        
        starts = pd.to_datetime(rows['forecast_start_date']).values.astype('datetime64[D]')
        ends = pd.to_datetime(rows['forecast_end_date']).values.astype('datetime64[D]')
        left_pos = np.searchsorted(rec_dates, starts, side='left')
        right_pos = np.searchsorted(rec_dates, ends, side='right') - 1

        sums = np.zeros(len(rows_index), dtype=float)
        # calculate sums
        for i, (l, r) in enumerate(zip(left_pos, right_pos)):
            if r >= l and r >= 0 and l < len(rec_cumulative):
                left_cum = rec_cumulative[l-1] if l > 0 else 0.0
                sums[i] = rec_cumulative[r] - left_cum
            else:
                sums[i] = 0.0
        y_train.loc[rows_index] = sums

    # drop of helper columns
    X_train_export = X_train.drop(columns=['forecast_start_date', 'forecast_end_date'], errors='ignore')
    # print("Training data and targets generated successfully.")
    return X_train_export, y_train

## Xgboost Functions

In [7]:
def _encode_rm_id_as_codes(X_train_full: pd.DataFrame, X_val: pd.DataFrame, X_test: pd.DataFrame):
    # rm_id categorical then map to stable int codes for XGBoost.

    if 'rm_id' not in X_train_full.columns:
        return X_train_full, X_val, X_test, []

    # print("Encoding rm_id as categorical codes  ")
    X_train_full = X_train_full.copy()
    X_train_full['rm_id'] = X_train_full['rm_id'].astype('category')
    rm_categories = X_train_full['rm_id'].cat.categories.tolist()

    def _align_categories(df):
        df = df.copy()
        df['rm_id'] = pd.Categorical(df['rm_id'], categories=rm_categories)
        df['rm_id'] = df['rm_id'].cat.codes.astype('int32') # Use codes for XGB
        return df

    X_val  = _align_categories(X_val)
    X_test = _align_categories(X_test)
    X_train_full['rm_id'] = X_train_full['rm_id'].cat.codes.astype('int32')

    return X_train_full, X_val, X_test, rm_categories

# Main 

## Data Preparation

In [8]:
print("Starting XGBoost Pipeline")
 
training_df = run_data_preparation()

# Separate target and weights (if they exist)
sample_weight_series = None
if 'recency_weight' in training_df.columns:
    sample_weight_series = training_df['recency_weight']

# Drop meta cols
drop_meta = [c for c in ['forecast_start_date', 'forecast_end_date', 'recency_weight'] 
             if c in training_df.columns]
X_train = training_df.drop(columns=['target'] + drop_meta, errors='ignore')
y_train = training_df['target']

print(f"Loaded training data: {X_train.shape[0]} rows, {X_train.shape[1]} features.")

Starting XGBoost Pipeline
Loaded training data: 552769 rows, 38 features.


## Test Set Creation

In [9]:

# build test set  
print("Loading raw data to build test set  ")
data_tuple = load_data()
if data_tuple[0] is None:
    raise FileNotFoundError("Raw data loading failed for test set creation.")

df_rec, purchase_orders, materials, prediction_mapping = data_tuple
daily_receivals, daily_po = aggregates_3df_daily(df_rec, purchase_orders, materials)

test_df_raw = prediction_mapping.set_index('ID')

X_test = create_features(
    test_df_raw, daily_receivals, daily_po,
    receivals_full=df_rec, purchase_orders_full=purchase_orders
)

# align test columns to train columns
X_test = X_test.reindex(columns=X_train.columns, fill_value=0.0)


Loading raw data to build test set  


## Validation Split

In [10]:
print("Creating chronological validation set (last 150 rows per rm_id)  ")
val_idx = X_train.groupby('rm_id').tail(150).index
X_val = X_train.loc[val_idx].copy()
y_val = y_train.loc[val_idx].copy()

train_idx = X_train.index.difference(val_idx)
X_train_full = X_train.loc[train_idx].copy()
y_train_full = y_train.loc[train_idx].copy()

# print(f"Training rows: {len(X_train_full)}, Validation rows: {len(X_val)}")

Creating chronological validation set (last 150 rows per rm_id)  


## Preprocessing for XGBoost

In [11]:

# encode rm_id as integer codes
X_train_full, X_val, X_test, rm_categories = _encode_rm_id_as_codes(X_train_full, X_val, X_test)

# sample weights
if sample_weight_series is None:
    sample_weight_series = pd.Series(1.0, index=X_train.index)
else:
    sample_weight_series = sample_weight_series.reindex(X_train.index).fillna(1.0)

sample_weight_val   = sample_weight_series.loc[val_idx].values
sample_weight_train = sample_weight_series.loc[train_idx].values

# QuantileDMatrix datasets
dtrain = xgb.QuantileDMatrix(X_train_full, y_train_full, weight=sample_weight_train)
dvalid = xgb.QuantileDMatrix(X_val, y_val, weight=sample_weight_val, ref=dtrain)
dtest  = xgb.QuantileDMatrix(X_test, ref=dtrain)

## Hyperparameter Tuning (Optuna)

In [12]:
### UNCOMMENT THE FOLLOWING TO ENABLE GPU USAGE ###
# it requires device= xgb_device in params!!

# use_gpu = os.system("nvidia-smi > /dev/null 2>&1") == 0
# if use_gpu:
#     print("\nGPU Rilevata! XGBoost userà 'cuda'.")
#     xgb_device = "cuda"
# else:
#     print("\nNessuna GPU rilevata (o driver non trovato). XGBoost userà la CPU.")
#     xgb_device = "cpu"


base_params = {
    "objective": "reg:quantileerror",
    "quantile_alpha": 0.2,
    "tree_method": "hist",
    "eval_metric": "quantile",
    "seed": SEED,
    "verbosity": 0,
}

best_params = {}
best_iter = None
best_score_for_log = None
n_trials_for_log = 0


print(f"Starting Optuna hyperparameter tuning ({HYPEROPT_N_TRIALS} trials)  ")

def objective(trial: "optuna.trial.Trial"):
    params = base_params.copy()
    params.update({
        "eta": trial.suggest_float("eta", 1e-3, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 64.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),
    })

    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=20000,
        evals=[(dtrain, "train"), (dvalid, "valid")],
        early_stopping_rounds=200,
        verbose_eval=False,
    )
    trial.set_user_attr("best_iteration", booster.best_iteration)
    trial.set_user_attr("best_score", booster.best_score)
    print(booster.attributes().get('scikit_learn', {}).get('device'))
    return float(booster.best_score)


study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=HYPEROPT_N_TRIALS, timeout=None)

best_params = study.best_params
best_iter = int(study.best_trial.user_attrs.get("best_iteration", 1000))
best_score_for_log = study.best_value
n_trials_for_log = len(study.trials)

print(f"Optuna found: best params: {best_params}, best score: {best_score_for_log}, in {n_trials_for_log} trials.")



[I 2025-11-09 13:20:21,595] A new study created in memory with name: no-name-66ff24f3-4444-450e-b738-d2c7777aafd0


Starting Optuna hyperparameter tuning (20 trials)  


[I 2025-11-09 13:22:32,349] Trial 0 finished with value: 14716.869282288972 and parameters: {'eta': 0.008468008575248327, 'max_depth': 12, 'min_child_weight': 6.110836758879259, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'reg_alpha': 2.5348407664333426e-07, 'reg_lambda': 3.3323645788192616e-08, 'gamma': 8.661761457749352}. Best is trial 0 with value: 14716.869282288972.


None


[I 2025-11-09 13:23:43,397] Trial 1 finished with value: 13999.656777253667 and parameters: {'eta': 0.030834348179355788, 'max_depth': 10, 'min_child_weight': 0.011977006629986053, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'reg_alpha': 8.148018307012941e-07, 'reg_lambda': 4.329370014459266e-07, 'gamma': 1.8340450985343382}. Best is trial 1 with value: 13999.656777253667.


...

None


[I 2025-11-09 13:42:24,488] Trial 18 finished with value: 13080.174776596517 and parameters: {'eta': 0.0035545030121162256, 'max_depth': 5, 'min_child_weight': 17.419208713304794, 'subsample': 0.90928355079006, 'colsample_bytree': 0.8214237152952776, 'reg_alpha': 2.1220102605209543e-05, 'reg_lambda': 0.004238580091270327, 'gamma': 8.257569030919115}. Best is trial 6 with value: 12867.018278318572.


None


[I 2025-11-09 13:43:30,410] Trial 19 finished with value: 13599.432224058028 and parameters: {'eta': 0.003423379766714161, 'max_depth': 5, 'min_child_weight': 18.50330376124951, 'subsample': 0.9329003770062791, 'colsample_bytree': 0.9512742826174833, 'reg_alpha': 1.3865505452822999e-05, 'reg_lambda': 0.23378225464218721, 'gamma': 8.15729837309543}. Best is trial 6 with value: 12867.018278318572.


None
Optuna found: best params: {'eta': 0.02260828676373495, 'max_depth': 4, 'min_child_weight': 49.024547442450555, 'subsample': 0.8875664116805573, 'colsample_bytree': 0.9697494707820946, 'reg_alpha': 1.1309571585271483, 'reg_lambda': 0.002404915432737351, 'gamma': 9.218742350231167}, best score: 12867.018278318572, in 20 trials.


## Final Model Training

In [13]:
print("Retraining final model on all training data (train + validation)")
X_combined = pd.concat([X_train_full, X_val], axis=0)
y_combined = pd.concat([y_train_full, y_val], axis=0)
sw_combined = pd.concat(
    [pd.Series(sample_weight_train, index=X_train_full.index),
     pd.Series(sample_weight_val, index=X_val.index)],
    axis=0
).reindex(X_combined.index).fillna(1.0).values

dcomb = xgb.QuantileDMatrix(X_combined, y_combined, weight=sw_combined)

final_params = base_params.copy()
final_params.update(best_params)

final_num_boost_round = (best_iter + 1 if best_iter is not None else 1000)
print(f"Training final model for {final_num_boost_round} iterations")

booster_final = xgb.train(
    params=final_params,
    dtrain=dcomb,
    num_boost_round=final_num_boost_round,
    verbose_eval=False,
)

print("Final model training complete.")

Retraining final model on all training data (train + validation)
Training final model for 375 iterations
Final model training complete.


## Analysis and Saving

Uncomment to plot feature importance, save model metadata and logs

In [14]:
# def plot_feature_importance(booster, feature_names, top_n=30):
#     print("\n  Feature Importance (Top 30 by 'Gain')  ")
#     importance = booster.get_score(importance_type='gain')
#     feature_map = {f'f{i}': name for i, name in enumerate(feature_names)}
#     importance = {feature_map.get(k, k): v for k, v in importance.items()}

#     importance_df = pd.DataFrame({
#         'feature': importance.keys(),
#         'importance': importance.values()
#     })
    
#     importance_df = importance_df.sort_values(by='importance', ascending=False).head(top_n)
    
#     plt.figure(figsize=(10, top_n / 2.5))
#     sns.barplot(x='importance', y='feature', data=importance_df, palette='viridis')
#     plt.title(f'Top {top_n} Feature Importances (Type: Gain)')
#     plt.xlabel('Importance (Gain)')
#     plt.ylabel('Feature')
#     plt.tight_layout()
#     plt.show()

# plot_feature_importance(booster_final, X_train_full.columns, top_n=30)


# print("Logging metrics to score_log.csv  ")
# log_file_path = '../score_log.csv'
# log_entry = {
#     "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#     "best_validation_score": best_score_for_log,
#     "best_iteration": best_iter,
#     "n_trials": n_trials_for_log,
#     "features_used": "|".join(X_train.columns.tolist()) 
# }
# log_df = pd.DataFrame([log_entry])


# if not os.path.exists(log_file_path):
#     log_df.to_csv(log_file_path, index=False)
# else:
#     log_df.to_csv(log_file_path, mode='a', header=False, index=False)
# print(f"Metrics successfully logged to {log_file_path}")

# model_file = '../xgb_model.json'
# meta_file  = '../xgb_model_meta.json'

# booster_final.save_model(str(model_file))

# meta = {
#     'feature_columns': X_train.columns.tolist(),
#     'categorical_rm_id_categories': rm_categories,
#     'quantile_alpha': 0.2,
#     'library': 'xgboost',
#     'best_iteration': int(best_iter) if best_iter is not None else None,
#     'best_params': best_params,
#     'best_validation_score': best_score_for_log
# }
# with open(meta_file, 'w', encoding='utf-8') as fh:
#     json.dump(meta, fh, indent=2)

# print(f"Final model saved to {model_file}")
# print(f"Metadata saved to {meta_file}")

## Prediction and Submission

In [15]:
print("Generating predictions on test set")

preds = booster_final.predict(dtest)
preds = np.asarray(preds, dtype=float)
preds[preds < 0] = 0.0  

submission = pd.DataFrame({'ID': X_test.index, 'predicted_weight': preds})
mapping_small = prediction_mapping[['ID', 'rm_id', 'forecast_end_date']].copy()
mapping_small['forecast_end_date'] = pd.to_datetime(mapping_small['forecast_end_date'])

merged_sub = submission.merge(mapping_small, on='ID', how='left').sort_values(['rm_id', 'forecast_end_date'])

merged_sub['predicted_weight'] = merged_sub.groupby('rm_id')['predicted_weight'].cummax()
submission = merged_sub.sort_values('ID')[['ID', 'predicted_weight']].reset_index(drop=True)

# Save submission
submission_filepath = '../final_submission_xgboost.csv'
submission.to_csv(submission_filepath, index=False)
print(f"Submission file saved to {submission_filepath}")

Generating predictions on test set
Submission file saved to ../final_submission_xgboost.csv
